In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# %pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

csv.field_size_limit(500 * 1024 * 1024)
CUDA_LAUNCH_BLOCKING=1


/usr/local/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv(r'dataset.csv', index_col=0, encoding='utf_8_sig')
df.fillna('', inplace=True)

In [3]:
df

,label,message,diff
0,negative,Merge pull request #46 from rufferson/saslx-tl...,diff --git a/lib/DJabberd.pm b/lib/DJabberd.pm...
1,positive,Fix leaks in kadmin server stubs [CVE-2015-863...,diff --git a/src/kadmin/server/server_stubs.c ...
2,positive,Validate authorization request on approval\n\n...,diff --git a/spring-security-oauth2/src/main/j...
3,positive,Release 2.72.4+171110,diff --git a/application/config/version.php b/...
4,negative,Fixing compiler warnings.,diff --git a/src/main.c b/src/main.c\nindex 50...
...,...,...,...
10107,negative,les: remove useless protocol defines (#22115)\...,diff --git a/les/benchmark.go b/les/benchmark....
10108,positive,Merge pull request #2067 from realm/tg-swift-l...,diff --git a/CHANGELOG.md b/CHANGELOG.md\ninde...
10109,negative,[fix] 新規アカウント作成ページの翻訳を修正,diff --git a/app/locales/ja_JP/LC_MESSAGES/mes...
10110,negative,Merge branch 'hotfixes',diff --git a/CHANGELOG.txt b/CHANGELOG.txt\nin...


In [4]:
df = df.rename(columns={'message':'text'})
train, test = train_test_split(df,test_size=0.96,stratify=df['label'],random_state=1)

In [5]:
from commitfit import get_templated_dataset,sample_dataset
from datasets import Dataset, load_metric

In [6]:
# train = df.rename(columns={'3_labels':'label','comment':'text'})

In [7]:
Dataset_train = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)


In [8]:
train_dataset = get_templated_dataset(Dataset_train, candidate_labels=['negative','positive'], sample_size=8)

In [9]:
# list(train['message'].astype(str).values)
train_dataset

Dataset({
    features: ['label', 'text', 'diff', '__index_level_0__'],
    num_rows: 420
})

In [10]:
len(train)

404

In [11]:
# encoded_train

In [12]:
train['label'].value_counts()

label
negative    254
positive    150
Name: count, dtype: int64

In [13]:
test['label'].value_counts()

label
negative    6093
positive    3615
Name: count, dtype: int64

In [14]:
# huggingface-cli login
# train_dataset = CommitDataset(encoded_train, list(train['3_labels']))
# test_dataset = CommitDataset(encoded_test, list(test['3_labels']))
# val_dataset = CommitDataset(encoded_val, list(val['label']))

In [15]:
len(train_dataset)

420

In [16]:
from sklearn import  metrics

def compute_metrics(y_pred, y_test):
    # print(y_pred,y_test)
    # classification_report = metrics.classification_report(y_test,y_pred)
    accuracy_score = metrics.accuracy_score(y_test,y_pred)
    precision_score = metrics.precision_score(y_test,y_pred, average='weighted')
    recall_score = metrics.recall_score(y_test,y_pred,average='weighted')
    f1_score = metrics.f1_score(y_test,y_pred,average='weighted')

    # return {"classification_report": classification_report}
    return {"accuracy": accuracy_score, "precision": precision_score,"recall": recall_score, "f1":f1_score}

In [17]:
train_dataset

Dataset({
    features: ['label', 'text', 'diff', '__index_level_0__'],
    num_rows: 420
})

In [18]:
model_id = "sentence-transformers/paraphrase-mpnet-base-v2"

In [19]:
from commitfit import CommitFitModel
model = CommitFitModel.from_pretrained(model_id)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [20]:
from commitfit import CommitFitTrainer

trainer = CommitFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    metric = compute_metrics,
    num_iterations=20,
    num_epochs=1
)

In [21]:
%%time
trainer.train()
fewshot_metrics = trainer.evaluate()
fewshot_metrics

Generating Training Pairs: 100%|██████████| 20/20 [00:00<00:00, 20.83it/s]
***** Running training *****
  Num examples = 16800
  Num epochs = 1
  Total optimization steps = 1050
  Total train batch size = 16
Epoch: 100%|██████████| 1/1 [09:34<00:00, 574.12s/it]
***** Running evaluation *****


CPU times: user 1h 4min 24s, sys: 34.2 s, total: 1h 4min 58s
Wall time: 10min


{'accuracy': 0.8262257931602802,
 'precision': 0.82820755611441,
 'recall': 0.8262257931602802,
 'f1': 0.8269718400367696}

In [22]:
import ipynbname
nb_fname = ipynbname.name()
print(nb_fname)

O-2-150


In [23]:
# store the checkpoints
trainer.model.save_pretrained(nb_fname)

In [24]:
# import os
# os.system('shutdown')

In [25]:
train.to_csv('train.csv',encoding='utf_8_sig')

In [26]:
test.to_csv('test.csv',encoding='utf_8_sig')